In [1]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep
import random

from datetime import datetime
from datetime import timedelta
import os
import time
from plotly.subplots import make_subplots
import glob


from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## 데이터 수집

In [3]:
ticker_nm = '005930'
ticker_nm = '247540'
start_date  = '20190101'
today_date1 = '20231001'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm

df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume','price_change_percentage', 'ticker']

In [4]:
ohlcv_df_raw = pd.read_csv('ohlcv_df_raw.csv', dtype = {'ticker':str})
ticker_list = ohlcv_df_raw['ticker'].unique()

In [5]:
ohlcv_df_raw

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name
0,2018-01-02,51380,51400,50780,51020,169485,0.117739,005930,삼성전자
1,2018-01-03,52540,52560,51420,51620,200270,1.176009,005930,삼성전자
2,2018-01-04,52120,52180,50640,51080,233909,-1.046106,005930,삼성전자
3,2018-01-05,51300,52120,51200,52120,189623,2.036022,005930,삼성전자
4,2018-01-08,52400,52520,51500,52020,167673,-0.191865,005930,삼성전자
...,...,...,...,...,...,...,...,...,...
130660,2023-09-26,41800,42200,40350,40600,119412,-5.140187,180640,한진칼
130661,2023-09-27,40200,42900,40200,42350,107323,4.310345,180640,한진칼
130662,2023-10-04,41750,42150,40200,40250,94897,-4.958678,180640,한진칼
130663,2023-10-05,40450,41200,40300,41200,41211,2.360248,180640,한진칼


In [9]:
train = ohlcv_df_raw
train

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name
0,2018-01-02,51380,51400,50780,51020,169485,0.117739,005930,삼성전자
1,2018-01-03,52540,52560,51420,51620,200270,1.176009,005930,삼성전자
2,2018-01-04,52120,52180,50640,51080,233909,-1.046106,005930,삼성전자
3,2018-01-05,51300,52120,51200,52120,189623,2.036022,005930,삼성전자
4,2018-01-08,52400,52520,51500,52020,167673,-0.191865,005930,삼성전자
...,...,...,...,...,...,...,...,...,...
130660,2023-09-26,41800,42200,40350,40600,119412,-5.140187,180640,한진칼
130661,2023-09-27,40200,42900,40200,42350,107323,4.310345,180640,한진칼
130662,2023-10-04,41750,42150,40200,40250,94897,-4.958678,180640,한진칼
130663,2023-10-05,40450,41200,40300,41200,41211,2.360248,180640,한진칼


In [10]:
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['ticker', 'final_return'])

In [11]:
results_df

,ticker,final_return


In [12]:
# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(ticker_list):
    # 학습 데이터 생성
    train_close = train[train['ticker'] == code][['date', 'close']]
    train_close['date'] = pd.to_datetime(train_close['date'], format='%Y-%m-%d')
    train_close.set_index('date', inplace=True)
    tc = train_close['close']
    
    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측
    
    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    
    # 결과 저장
    result_sample = pd.DataFrame({'ticker': code, 'final_return': final_return}, index = [0])
#     results_df = results_df.append({'ticker': code, 'final_return': final_return}, ignore_index=True)
    results_df = pd.concat([results_df, result_sample])

  0%|          | 0/100 [00:00<?, ?it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters f

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
  6%|▌         | 6/100 [00:03<01:03,  1.48it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequ

 13%|█▎        | 13/100 [00:10<01:10,  1.24it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction r

 21%|██        | 21/100 [00:16<00:52,  1.51it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive par

 28%|██▊       | 28/100 [00:19<00:39,  1.82it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction r

...

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
 65%|██████▌   | 65/100 [00:48<00:22,  1.56it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated freq

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
 72%|███████▏  | 72/100 [00:54<00:22,  1.27it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated freq

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
 79%|███████▉  | 79/100 [00:58<00:13,  1.56it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so wil

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
 86%|████████▌ | 86/100 [01:04<00:10,  1.34it/s]/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/shjj08choi/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated freq

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [23]:
results_df

,ticker,final_return
0,005930,0.001683
0,373220,-0.000604
0,000660,0.000001
0,207940,-0.000107
0,005935,-0.000059
...,...,...
0,006800,0.000532
0,078930,0.000236
0,251270,0.000933
0,029780,0.003426


In [24]:
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성


In [35]:
public = ohlcv_df_raw[ohlcv_df_raw['date'] <= '2023-05-30']
private = ohlcv_df_raw
public

,date,open,high,low,close,volume,price_change_percentage,ticker,corp_name
0,2018-01-02,51380,51400,50780,51020,169485,0.117739,005930,삼성전자
1,2018-01-03,52540,52560,51420,51620,200270,1.176009,005930,삼성전자
2,2018-01-04,52120,52180,50640,51080,233909,-1.046106,005930,삼성전자
3,2018-01-05,51300,52120,51200,52120,189623,2.036022,005930,삼성전자
4,2018-01-08,52400,52520,51500,52020,167673,-0.191865,005930,삼성전자
...,...,...,...,...,...,...,...,...,...
130573,2023-05-23,45000,45100,44050,44600,46054,0.000000,180640,한진칼
130574,2023-05-24,44300,44550,43450,43900,41447,-1.569507,180640,한진칼
130575,2023-05-25,43150,43700,42250,42350,64579,-3.530752,180640,한진칼
130576,2023-05-26,41950,42700,41750,42200,33884,-0.354191,180640,한진칼


In [41]:
results_df = pd.DataFrame()
# 15일부터 60일까지 5일 간격으로 window size를 변경하면서 계산 반복
for i in range(15,65,5):
    results_df = pd.DataFrame(columns=['ticker', 'Sharpe'])
    for code in tqdm(ticker_list):
        public_close = public[public['ticker'] == code][['close','price_change_percentage']]
        public_close_15 = public_close[-i:]
        final_return = (public_close_15['close'].iloc[-1] - public_close_15['close'].iloc[0]) / public_close_15['close'].iloc[0]
        std_deviation = np.std(public_close_15['price_change_percentage'])
        sharpe_ratio = final_return/std_deviation
        
#         results_df = results_df.append({'ticker': code, 'Sharpe': float(sharpe_ratio)}, ignore_index=True)
        result_sample = pd.DataFrame({'ticker': code, 'final_return': final_return}, index = [0])
#         results_df = results_df.append({'ticker': code, 'final_return': final_return}, ignore_index=True)
        results_df = pd.concat([results_df, result_sample])
    
    # window size별 샤프지수 순위 저장
    results_df = results_df.fillna(0)
    results_df['순위'] = results_df['ticker'].rank(method='first', ascending=False).astype('int')
    sample_submission = pd.read_csv('sample_submission.csv')
    baseline_submission = sample_submission[['ticker']].merge(results_df[['ticker', '순위']], on='ticker', how='left')
    
    # 샤프지수 산출식을 사용한 window size별 샤프지수 검증
    public_test = private.loc['2023-05-31' : '2023-06-21']
    close_dict = public_test.sort_values(by='Date').groupby(by='ticker')['close'].apply(list).to_dict()
    rank_list = baseline_submission.sort_values(by='순위')['ticker'].tolist()

    buy_codes = rank_list[:200]    # 상위 200개 매수
    sell_codes = rank_list[-200:]  # 하위 200개 공매도
    buy_prices = np.array([close_dict[cc] for cc in buy_codes])
    sell_prices = np.array([close_dict[cc] for cc in sell_codes])

    assert buy_prices.shape[1] == 15

    sharpe = {}
    sharpe['일수'] = 15
    sharpe['무위험이자율'] = 0.035
    total_buy_returns = np.sum(buy_prices[:, -1] / buy_prices[:, 0] - 1.0)
    total_sell_returns = -np.sum(sell_prices[:, -1] / sell_prices[:, 0] - 1.0)
    sharpe['매수 수익률'] = total_buy_returns
    sharpe['공매도 수익률'] = total_sell_returns
    sharpe['최종 수익률'] = (total_buy_returns + total_sell_returns) / 400
    sharpe['연율화된 최종 수익률'] = sharpe['최종 수익률'] * 250 / sharpe['일수']
    buy_daily_profits = +(buy_prices[:, 1:] / buy_prices[:, :-1] - 1.0)
    sell_daily_profits = -(sell_prices[:, 1:] / sell_prices[:, :-1] - 1.0)
    sharpe['연율화된 수익률의 일별 평균'] = (buy_daily_profits + sell_daily_profits).sum(0) * 250 / 400
    sharpe['연율화된 수익률의 평균'] = np.mean(sharpe['연율화된 수익률의 일별 평균'])
    daily_volatility = np.sum((sharpe['연율화된 수익률의 일별 평균'] - sharpe['연율화된 수익률의 평균']) ** 2.0) / (15 - 2)
    sharpe['최종 수익률 변동성'] = np.sqrt(daily_volatility)
    sharpe['샤프지수'] = (sharpe['연율화된 최종 수익률'] - sharpe['무위험이자율']) / sharpe['최종 수익률 변동성']
    
    print(sharpe['샤프지수'])

 88%|████████▊ | 88/100 [00:01<00:00, 69.98it/s]


IndexError: single positional indexer is out-of-bounds